In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '2'
import numpy as np
import pandas as pd
import tensorflow as tf

import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
%matplotlib inline

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
import os
import time
import numpy as np
import tensorflow as tf

class GAN():
    def __init__(self):
        super().__init__()

    def load(self, npy_path):
        self.data_dict = np.load(npy_path, encoding='latin1').item()
        print("Load %s as self.data_dict" % npy_path)

    def build(self, n_dim=512, shape=(64,64,3)):
        """
        load pre-trained weights from path
        :param vgg16_npy_path: file path of vgg16 pre-trained weights
        """
        # input information
        self.H, self.W, self.C = shape
        self.n_dim = n_dim
        
        # parameter dictionary
        self.para_dict = dict()
        self.data_dict = dict()
        self.net_shape = dict()

        # input placeholder
        self.x = tf.placeholder(tf.float32, [None, self.H, self.W, self.C])
        self.is_train = tf.placeholder(tf.bool)
        self.random_sample = tf.placeholder(tf.float32, [None, self.n_dim])
        
        # normalize inputs
        assert self.x.get_shape().as_list()[1:] == [self.H, self.W, self.C]
        

        with tf.variable_scope("Generator",reuse=tf.AUTO_REUSE):
            self.G_image = self.generator(self.random_sample)
            
        with tf.variable_scope("Discriminator", reuse=tf.AUTO_REUSE):
            self.D_real = self.discriminator(self.x)
            self.D_fake = self.discriminator(self.G_image)
    
        self.real_equality = tf.equal(tf.cast(tf.sigmoid(self.D_real) > 0.5, tf.float32), tf.ones(shape=tf.shape(self.D_real)))
        self.D_real_accu = tf.reduce_mean(tf.cast(self.real_equality, tf.float32))
        
        self.fake_equality = tf.equal(tf.cast(tf.sigmoid(self.D_fake) > 0.5, tf.float32), tf.zeros(shape=tf.shape(self.D_fake)))
        self.D_fake_accu = tf.reduce_mean(tf.cast(self.fake_equality, tf.float32))
        
        # loss of discriminator
        self.D_real_loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=self.D_real,
                                                                                  labels=tf.ones(shape=tf.shape(self.D_real))))
        self.D_fake_loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=self.D_fake,
                                                                                  labels=tf.zeros(shape=tf.shape(self.D_fake))))
        
        self.D_loss = self.D_real_loss + self.D_fake_loss
        
        # loss of generator
        self.G_loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=self.D_fake,
                                                                             labels=tf.ones(shape=tf.shape(self.D_fake))))
    
    def discriminator(self, input_image):
        conv1 = self.conv_bn_layer(input_image, shape=(4,4,3,32), stride=2, name="conv1")
        conv2 = self.conv_bn_layer(conv1 , shape=(4,4,32,64), stride=2, name="conv2")
        conv3 = self.conv_bn_layer(conv2 , shape=(4,4,64,128), stride=2, name="conv3")
        conv4 = self.conv_bn_layer(conv3 , shape=(4,4,128,256), stride=2, name="conv4")
        flatten = self.flatten_layer(conv4, name='flatten')
        output = self.dense_layer(flatten, n_hidden=1, name='D_output')
        return output
        
    def generator(self, sample_input):
        deconv_fc1 = self.dense_layer(sample_input, n_hidden=4096, name='deconv_fc1')
        deconv_input = tf.reshape(deconv_fc1, shape=[-1, 4, 4, 256])
        batch_size = tf.shape(sample_input)[0]
        deconv1 = self.trans_conv_layer(bottom=deconv_input, shape=(4,4,128,256),
                                        output_shape=[batch_size, 8, 8, 128], stride=2, name='deconv1')
        deconv2 = self.trans_conv_layer(bottom=deconv1, shape=(4,4,64,128),
                                        output_shape=[batch_size, 16, 16, 64], stride=2, name='deconv2')
        deconv3 = self.trans_conv_layer(bottom=deconv2, shape=(4,4,32,64),
                                        output_shape=[batch_size, 32, 32, 32], stride=2, name='deconv3')
        output = self.trans_conv_layer(bottom=deconv3, shape=(4,4,3,32),
                                        output_shape=[batch_size, self.H, self.W, self.C], activation='tanh', stride=2, name='deconv_output')
        return (output/2) + 0.5

    def dense_layer(self, bottom, n_hidden=None, name=None):
        bottom_shape = bottom.get_shape().as_list()
        if n_hidden is not None:
            W = self.get_weights(shape=(bottom_shape[1], n_hidden), name=name)
            b = self.get_bias(shape=n_hidden, name=name)
        elif name in self.data_dict.keys():
            W = self.get_weights(name=name)
            b = self.get_bias(name=name)
        else:
            print("Neither give a shape nor lack a pre-trained layer called %s" % name)
        self.para_dict[name] = [W, b]
        fc = tf.nn.bias_add(tf.matmul(bottom, W), b)
        self.net_shape[name] = fc.get_shape().as_list()
        return fc

    def flatten_layer(self, bottom, name):
        shape = bottom.get_shape().as_list()
        dim = 1
        for d in shape[1:]:
            dim *= d
        flatten = tf.reshape(bottom, [-1, dim])
        self.net_shape[name] = flatten.get_shape().as_list()
        return flatten

    def avg_pool_layer(self, bottom, name):
        pool = tf.nn.avg_pool(bottom, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME', name=name)
        self.net_shape[name] = pool.get_shape().as_list()
        return pool

    def max_pool_layer(self, bottom, name):
        pool = tf.nn.max_pool(bottom, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME', name=name)
        self.net_shape[name] = pool.get_shape().as_list()
        return pool
    
    def dropout(self, bottom, keep_prob):
        if self.is_train == True:
            return tf.nn.dropout(bottom, keep_prob=keep_prob)
        else:
            return bottom

    def trans_conv_layer(self, bottom, output_shape, stride, activation='relu', name=None, shape=None):
        if shape is not None:
            conv_filter, gamma, beta, bn_mean, bn_variance = self.get_conv_filter(shape=shape, name=name)
            conv_bias = self.get_bias(shape=shape[2], name=name)
        elif name in self.data_dict.keys():
            conv_filter, gamma, beta, bn_mean, bn_variance = self.get_conv_filter(name=name)
            conv_bias = self.get_bias(name=name)
        else:
            print("Neither give a shape nor lack a pre-trained layer called %s" % name)

        self.para_dict[name] = [conv_filter, conv_bias]
        self.para_dict[name+"_gamma"] = gamma
        self.para_dict[name+"_beta"] = beta
        self.para_dict[name+"_bn_mean"] = bn_mean
        self.para_dict[name+"_bn_variance"] = bn_variance

        conv = tf.nn.conv2d_transpose(bottom, conv_filter, output_shape, strides=[1, stride, stride, 1], padding="SAME")
        conv = tf.nn.bias_add(conv, conv_bias)
        
        from tensorflow.python.training.moving_averages import assign_moving_average
        def mean_var_with_update():
            mean, variance = tf.nn.moments(conv, [0,1,2], name='moments')
            with tf.control_dependencies([assign_moving_average(bn_mean, mean, 0.99),
                                            assign_moving_average(bn_variance, variance, 0.99)]):
                return tf.identity(mean), tf.identity(variance)

        mean, variance = tf.cond(self.is_train, mean_var_with_update, lambda:(bn_mean, bn_variance))
        conv = tf.nn.batch_normalization(conv, mean, variance, beta, gamma, 1e-05)
        self.net_shape[name] = conv.get_shape().as_list()

        if activation=='tanh':
            print('tanh')
            tanh = tf.nn.tanh(conv)
            return tanh
        else:
            relu = tf.nn.leaky_relu(conv)
            return relu

    def conv_bn_layer(self, bottom, stride=1, activation='lrelu', name=None, shape=None):
        if shape is not None:
            conv_filter, gamma, beta, bn_mean, bn_variance = self.get_conv_filter(shape=shape, name=name)
            conv_bias = self.get_bias(shape=shape[3], name=name)
        elif name in self.data_dict.keys():
            conv_filter, gamma, beta, bn_mean, bn_variance = self.get_conv_filter(name=name)
            conv_bias = self.get_bias(name=name)
        else:
            print("Neither give a shape nor lack a pre-trained layer called %s" % name)

        self.para_dict[name] = [conv_filter, conv_bias]
        self.para_dict[name+"_gamma"] = gamma
        self.para_dict[name+"_beta"] = beta
        self.para_dict[name+"_bn_mean"] = bn_mean
        self.para_dict[name+"_bn_variance"] = bn_variance

        conv = tf.nn.conv2d(bottom, conv_filter, [1, stride, stride, 1], padding='SAME')
        conv = tf.nn.bias_add(conv, conv_bias)

        from tensorflow.python.training.moving_averages import assign_moving_average
        def mean_var_with_update():
            mean, variance = tf.nn.moments(conv, [0,1,2], name='moments')
            with tf.control_dependencies([assign_moving_average(bn_mean, mean, 0.99),
                                            assign_moving_average(bn_variance, variance, 0.99)]):
                return tf.identity(mean), tf.identity(variance)

        mean, variance = tf.cond(self.is_train, mean_var_with_update, lambda:(bn_mean, bn_variance))

        conv = tf.nn.batch_normalization(conv, mean, variance, beta, gamma, 1e-05)
        self.net_shape[name] = conv.get_shape().as_list()

        if activation=='tanh':
            tanh = tf.nn.tanh(conv)
            return tanh
        else:
            relu = tf.nn.leaky_relu(conv)
            return relu

    def get_conv_filter(self, shape=None, name=None, with_bn=True):
        if shape is not None:
            conv_filter = tf.get_variable(shape=shape, initializer=tf.truncated_normal_initializer(mean=0, stddev=0.1), name=name+"_W", dtype=tf.float32)
        elif name in self.data_dict.keys():
            conv_filter = tf.get_variable(initializer=self.data_dict[name][0], name=name+"_W")
        else:
            print("Neither give a shape nor lack a pre-trained layer called %s" % name)
            return None

        if with_bn:
            if 'deconv' in name:
                H,W,O,C = conv_filter.get_shape().as_list()
            else:
                H,W,C,O = conv_filter.get_shape().as_list()

            if name+"_gamma" in self.data_dict.keys(): 
                gamma = tf.get_variable(initializer=self.data_dict[name+"_gamma"], name=name+"_gamma")
            else:
                gamma = tf.get_variable(shape=(O,), initializer=tf.ones_initializer(), name=name+"_gamma")

            if name+"_beta" in self.data_dict.keys(): 
                beta = tf.get_variable(initializer=self.data_dict[name+"_beta"], name=name+"_beta")
            else:
                beta = tf.get_variable(shape=(O,), initializer=tf.zeros_initializer(), name=name+'_beta')

            if name+"_bn_mean" in self.data_dict.keys(): 
                bn_mean = tf.get_variable(initializer=self.data_dict[name+"_bn_mean"], name=name+"_bn_mean")
            else:
                bn_mean = tf.get_variable(shape=(O,), initializer=tf.zeros_initializer(), name=name+'_bn_mean')

            if name+"_bn_variance" in self.data_dict.keys(): 
                bn_variance = tf.get_variable(initializer=self.data_dict[name+"_bn_variance"], name=name+"_bn_variance")
            else:
                bn_variance = tf.get_variable(shape=(O,), initializer=tf.ones_initializer(), name=name+'_bn_variance')
            return conv_filter, gamma, beta, bn_mean, bn_variance
        else:
            return conv_filter
    
    def get_weights(self, shape=None, name=None):
        if shape is not None:
            return tf.get_variable(shape=shape, initializer=tf.truncated_normal_initializer(mean=0, stddev=0.1), name=name+"_W", dtype=tf.float32)
        elif name in self.data_dict.keys(): 
            return tf.get_variable(initializer=self.data_dict[name][0], name=name+"_W")
        else:
            print("(get_weight) neither give a shape nor lack a pre-trained layer called %s" % name)
            return None
            
    def get_bias(self, shape=None, name=None):
        if shape is not None:
            return tf.get_variable(shape=shape, initializer=tf.truncated_normal_initializer(mean=0, stddev=0.1), name=name+"_b", dtype=tf.float32)
        elif name in self.data_dict.keys(): 
            return tf.get_variable(initializer=self.data_dict[name][1], name=name+"_b")
        else:
            print("(get_bias) neither give a shape nor lack a pre-trained layer called %s" % name)
            return None


In [3]:
# %load train.py
import os
import time
import argparse
import numpy as np
import tensorflow as tf

from progress.bar import Bar
from ipywidgets import IntProgress
from IPython.display import display
import skimage.transform
import imageio

from utils import read_dataset

TRAIN_CSV = "hw4_data/train.csv"
TRAIN_DIR = "hw4_data/train/"
TEST_CSV = "hw4_data/test.csv"
TEST_DIR = "hw4_data/test/"

FLAG_lr = 1e-4
FLAG_save_dir = 'save_gan_n1/'
FLAG_batch_size = 32
FLAG_n_dim = 100
FLAG_D_G_ratio = 2

if not os.path.exists(FLAG_save_dir):
    print("Creating %s" % FLAG_save_dir)
    os.makedirs(FLAG_save_dir)
if not os.path.exists(os.path.join(FLAG_save_dir, "recons")):
    print("Creating %s" % os.path.join(FLAG_save_dir, "recons"))
    os.makedirs(os.path.join(FLAG_save_dir, "recons"))
if not os.path.exists(os.path.join(FLAG_save_dir, "samples")):
    print("Creating %s" % os.path.join(FLAG_save_dir, "samples"))
    os.makedirs(os.path.join(FLAG_save_dir, "samples"))

print("Reading dataset...")
# load data
Xtrain, df_train = read_dataset(TRAIN_CSV, TRAIN_DIR)
Xtest , df_test  = read_dataset(TEST_CSV , TEST_DIR)

gan = GAN()
gan.build(n_dim=FLAG_n_dim, shape=Xtrain.shape[1:])

saver = tf.train.Saver(tf.global_variables(), max_to_keep=None)

def initialize_uninitialized(sess):
    global_vars = tf.global_variables()
    is_not_initialized = sess.run([tf.is_variable_initialized(var) for var in global_vars])
    not_initialized_vars = [v for (v,f) in zip(global_vars, is_not_initialized) if not f]
    if len(not_initialized_vars): 
        sess.run(tf.variables_initializer(not_initialized_vars))

def res_plot(samples, n_row, n_col):     
    fig = plt.figure(figsize=(n_col*2, n_row*2))
    gs = gridspec.GridSpec(n_row, n_col)
    gs.update(wspace=0.05, hspace=0.05)
    for i, sample in enumerate(samples):
        ax = plt.subplot(gs[i])
        plt.axis('off')
        ax.set_xticklabels([])
        ax.set_yticklabels([])
        ax.set_aspect('equal')
        plt.imshow(sample.reshape(64, 64, 3))
    return fig

Creating save_gan_n1/
Creating save_gan_n1/recons
Creating save_gan_n1/samples
Reading dataset...
tanh


In [ ]:
tvars = tf.trainable_variables()
D_vars, G_vars = [],[]
for v in tvars:
    if 'Generator' in v.name:
        G_vars.append(v)
    if 'Discriminator' in v.name:
        D_vars.append(v)

In [ ]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())

    # hyper parameters
    batch_size = FLAG_batch_size
    epoch = 50
    min_delta = 0.0001

    # recorder
    epoch_counter = 0

    # optimizer
    global_step = tf.Variable(0, trainable=False)

    # Passing global_step to minimize() will increment it at each step.
    start_learning_rate = FLAG_lr
    half_cycle = 12500
    learning_rate = tf.train.exponential_decay(start_learning_rate, global_step, half_cycle, 0.5, staircase=True)
    
    # define D and G operation, with respect to their own variabls
    D_train_op = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(gan.D_loss, global_step=global_step, var_list=D_vars)
    G_train_op = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(gan.G_loss, global_step=global_step, var_list=G_vars)    

    # progress bar
    ptrain = IntProgress()
    display(ptrain)
    ptrain.max = int(Xtrain.shape[0]/batch_size)

    # re-initialize
    initialize_uninitialized(sess)

    loss_dict = dict({'D_fake_accu':[], 
                      'D_real_accu':[],
                      'G_loss':[]
                     })
    
    
    # optimize when the aggregated obj
    while(epoch_counter < epoch):

        # start training
        stime = time.time()
        bar_train = Bar('Training', max=int(Xtrain.shape[0]/batch_size), suffix='%(index)d/%(max)d - %(percent).1f%% - %(eta)ds')
        bar_val =  Bar('Validation', max=int(Xtest.shape[0]/batch_size), suffix='%(index)d/%(max)d - %(percent).1f%% - %(eta)ds')

        epoch_D_real_accu = 0.0
        epoch_D_fake_accu = 0.0
        epoch_G_loss = 0.0
        stime = time.time()
        for i in range(int(Xtrain.shape[0]/batch_size)):
            # Update D
            st = i*batch_size
            ed = (i+1)*batch_size
            real_accu, fake_accu, _ = sess.run([gan.D_real_accu,gan.D_fake_accu,D_train_op],
                               feed_dict={gan.x: Xtrain[st:ed,:],
                                          gan.random_sample: np.random.uniform(-1, 1, [batch_size, gan.n_dim]).astype(np.float32),
                                          gan.is_train: True}
                              )
            epoch_D_real_accu += np.mean(real_accu)
            epoch_D_fake_accu += np.mean(fake_accu)
        
            # Update G
            for _ in range(FLAG_D_G_ratio):
                loss, _ = sess.run([gan.G_loss,G_train_op],
                                   feed_dict={gan.x: Xtrain[st:ed,:],
                                              gan.random_sample: np.random.uniform(-1, 1, [batch_size, gan.n_dim]).astype(np.float32),
                                              gan.is_train: True})
                epoch_G_loss += np.mean(loss)
            ptrain.value += 1
            ptrain.description = "G-Training %s/%s" % (ptrain.value, ptrain.max)
            
        print("Epoch %s, %s sec >> train D real accu: %.4f, train D fake accu: %.4f, train G loss: %.4f" % (epoch_counter, round(time.time()-stime,2), epoch_D_real_accu/ptrain.value, epoch_D_fake_accu/ptrain.value, epoch_G_loss/ptrain.value))    
        loss_dict['D_real_accu'].append(epoch_D_real_accu/ptrain.value)
        loss_dict['D_fake_accu'].append(epoch_D_fake_accu/ptrain.value)
        loss_dict['G_loss'].append(epoch_G_loss/ptrain.value)
        
        # epoch end
        # shuffle Xtrain and Ytrain in the next epoch
        idx = np.random.permutation(Xtrain.shape[0])
        Xtrain= Xtrain[idx,:,:,:]
        ptrain.value = 0
        bar_train.finish()
        epoch_counter += 1

            
        # plot
        Xplot = sess.run(gan.G_image,
                feed_dict={gan.random_sample: np.random.uniform(-1, 1, [batch_size, gan.n_dim]).astype(np.float32),
                           gan.is_train: False})
        fig = res_plot(Xplot, int(batch_size/8), 8)
        plt.savefig(os.path.join(FLAG_save_dir, 'samples', '{}.png'.format(str(epoch_counter).zfill(3))), 
                    bbox_inches='tight')
        plt.close(fig)
        
        if epoch_counter%3==0 and epoch_counter>20:
            saver.save(sess,os.path.join(FLAG_save_dir,'model.ckpt'),global_step=epoch_counter)
np.save(os.path.join(FLAG_save_dir, "history_dict.npy"), loss_dict)           

IntProgress(value=0)

Epoch 0, 78.32 sec >> train D real accu: 0.9048, train D fake accu: 0.9226, train G loss: 8.3313
Epoch 1, 75.78 sec >> train D real accu: 0.8028, train D fake accu: 0.8327, train G loss: 5.6123
Epoch 2, 76.18 sec >> train D real accu: 0.7083, train D fake accu: 0.7452, train G loss: 4.2717
Epoch 3, 76.68 sec >> train D real accu: 0.7111, train D fake accu: 0.7560, train G loss: 3.6873
Epoch 4, 76.68 sec >> train D real accu: 0.6883, train D fake accu: 0.7319, train G loss: 3.0329
Epoch 5, 76.37 sec >> train D real accu: 0.6448, train D fake accu: 0.6792, train G loss: 2.5456
Epoch 6, 76.43 sec >> train D real accu: 0.6163, train D fake accu: 0.6420, train G loss: 2.2537
Epoch 7, 76.01 sec >> train D real accu: 0.6109, train D fake accu: 0.6353, train G loss: 1.9851
Epoch 8, 76.25 sec >> train D real accu: 0.5842, train D fake accu: 0.6020, train G loss: 1.8384


In [ ]:
loss_dict

In [ ]:
d = np.load("save_gan_v2/history_dict.npy", encoding='latin1').item()

In [ ]:
a= pd.DataFrame.from_dict(d)

In [ ]:
ax = a[['D_fake_accu', 'D_real_accu']].plot()
ax.set_xlabel('epoch')
ax.set_ylabel('accuarcy')
ax.set_title('GAN Discriminator')

In [ ]:
ax = a['G_loss'].plot()
ax.set_xlabel('epoch')
ax.set_ylabel('loss')
ax.set_title('GAN Generator')